# 4bit quantization으로 Polyglot-ko 12.8B QLoRA 학습하기

- original model : EleutherAI/polyglot-ko-12.8b -> polyglot-ko-12.8b를 1GB로 smaller하게 만든 모델
- EleutherAI/polyglot-ko-12.8b : GPT-NeoX framework

In [1]:
!nvidia-smi

Thu Feb 15 03:26:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are insta

## 크롤링 데이터 셋 json 파일로 로딩

In [3]:
from datasets import load_dataset

# 사용자 정의 JSON 파일 경로
custom_json_file_path = "/content/blog_data_new.json"

# 사용자 정의 데이터셋 로드
custom_dataset = load_dataset("json", data_files=custom_json_file_path)

# HuggingFace의 datasets 형태로 json 파일을 로드

# 데이터셋 확인
print(custom_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 538
    })
})


In [4]:
# json 파일의 내용을 학습에 위한 형태로 변환하기 위해 mapping

tr_data = custom_dataset.map(
    lambda x: {'text': f"### 역할: {x['instruction']}\n\n###작성하고 싶은 블로그 내용: {x['input']}\n\n### 블로그 본문: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

## 모델 로드

- 원래는 단일 파일이기도 하지만, 작은 파일(약 1GB)로 쪼개서 개별로 로드한 레포를 쓰면 RAM이 안터짐
- 해당 부분에서 model_id를 원하는 다른 model_id로 변경 시 다른 모델로 fine tune 가능

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-5.8b"  # model_id 원하는 모델 불러오기
# EleutherAI/polyglot-ko-5.8b -> polyglot - ko
# EleutherAI/polyglot-ko-12.8b -> polyglot - ko
# beomi/KoAlpaca-13B-LoRA -> KoAlpaca
#

# 양자화 진행
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

...

model-00011-of-00013.safetensors:   0%|          | 0.00/952M [00:00<?, ?B/s]

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## 텍스트 데이터만 tokenize

In [7]:
data = tr_data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/538 [00:00<?, ? examples/s]

In [8]:
data['train'][0]['text']

'### 역할: 당신은 인공지능 블로그 작성자입니다. 사용자가 여러분에게 주제를 줍니다. 당신의 목표는 주어진 주제를 바탕으로 블로그 내용을 작성하는 것입니다. 블로그를 작성하는 동안 다양한 생각과 여러가지 방식을 고려하여 500자 이상의 블로그를 작성하세요 .\n\n###작성하고 싶은 블로그 내용: 여행 터키 일주일 자유 여행 소울투어터키 패키지 솔직 후기\n\n### 블로그 본문: \u200b \u200b직장인의 일주일 터키 자유 여행굉장히 한정된 시간속에서 넓디 넓은튀르키예라는 나라를 여행하기란 정말 벅찬 일이다그렇게 가게 된 세미 패키지 여행\u200b나처럼 시간은 여유롭지 못하고보고 싶은 곳은 많다면세미 패키지도 좋은 대안이 될 수 있다아무래도 튀르키예는 이동시간만 엄청나게 잡아먹기 때문이다\u200b  📍이 포스팅엔 이런 내용이 있어요\u200b\u200b1. 터키 자유 여행 세미패키지 결정 이유2. 소울투어 터키의 솔직 후기3. 가격, 일정, 꿀팁\u200b예약 링크, 이스탄불 일일투어 후기 포스팅링크도 첨부했습니다 :)  \u200b\u200b 터키 5박 6일 지방투어 소울터키\u200b나는 일주일, 총 7박 9일의 시간이 있었고그 중 앞뒤 2박은 이스탄불에서 자유 여행을그리고 중간 5박은 세미패키지를 통해 여행했다\u200b\u200b  \u200b아예 항공권까지 포함한 패키지 상품이 많은데소울투어 세미패키지는세미패키지인만큼 항공권은 별도!\u200b마이리얼트립에서 인기 좋은소울투어터키 라는 곳에서 예약하게 되었다\u200b\u200b\u200b 왜 세미패키지를 결정하게 되었을까? \u200b비용과 시간을 효율적이고 합리적으로사용할 수 있다는 점그리고 이동이 편리함 점계획을 굳이 세우지 않아도 된다는 점  \u200b여행을 다녀 온 후에도 아주 자신있게 말할 수 있지만위 장점들은 사실(?)이였다생각했던 장점에 따른 결정에 대해후회는 없었다 :)\u200b나도 일일투어만 해봤지이런 패키지 여행은 몽골 이후로 처음인데생각보다 알차고 편해서 놀랐다 

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비해줍시다.
- QLoRA 사용 코드

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
# QLoRA를 통해 감소된 학습 파라미터 수를 보여주는 함수
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

QLoRA의 Config 설정 및 QLoRA 적용 후의 파라미터 수 표현

In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


In [12]:
!nvidia-smi

Thu Feb 15 04:04:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              39W / 300W |   4720MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 학습

- 100개 스텝만 학습

In [13]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=100, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=5e-5,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:437: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.275000
20,2.557500
30,2.202900
40,2.195200
50,2.049800
60,2.113400
70,2.195300
80,2.254400
90,1.996700
100,1.996600


TrainOutput(global_step=100, training_loss=2.183664531707764, metrics={'train_runtime': 272.2509, 'train_samples_per_second': 0.735, 'train_steps_per_second': 0.367, 'total_flos': 1.1113526817718272e+16, 'train_loss': 2.183664531707764, 'epoch': 0.37})

In [14]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [15]:
model.generate(**tokenizer("### 작성하고 싶은 블로그 내용: 일본 여행에 대해서 블로그를 작성해줘", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1155: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1447: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 3601, 4296, 1219,  296, 9189, 1398,   29, 1191, 2213,
          274, 1626, 9189,  301, 3601,  310, 3219,   17]])

In [17]:
# 모델의 결과물을 출력하는 함수 작성 #

def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 작성하고 싶은 블로그 내용: {x}\n\n### 블로그 본문:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 실제 블로그 생성 AI 실행

In [18]:
gen('일본 여행에 대해서 블로그를 작성해줘')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 작성하고 싶은 블로그 내용: 일본 여행에 대해서 블로그를 작성해줘

### 블로그 본문:### 블로그 댓글: ##### 블로그 본문:####################################################### #일본 후쿠오카여행 #유유버스 타고 #여행하자 #여행코스 #버스여행 #여행예약​안녕하세요~ 이웃님들^^​ ​이제 슬슬 여름 성수기라국내여행부터 해외여행까지준비 많이 하시는 시기죠?​저도 올해 해외여행 계획하면서지난 번 오사카 여행때 이용했던유유버스를 다시 이용하게 되어여러분들께 소개해드리려고 해요~​그럼 여행코스부터예약팁까지 자세히 알려드릴게요^^​ ​​ 후쿠오카 여행코스 알아보기​먼저 후쿠오카는 한국과 가까워서 비행시간도 짧아비행기로 다녀오기 괜찮은 곳이에요​저도 후쿠오카로 갈 때는 비행기,한국으로 돌아올때는 배타고 왔어요​저는 후쿠오카 공항-유후인-뱃부-유후인​이렇게 총 4일 일정이었는데요보통


- 50step 학습

`### 작성하고 싶은 블로그 내용: 일본 여행에 대해서 블로그를 작성해줘`

`### 블로그 본문: 일본여행은 2박3일로 친구와 계획중입니다저는 일본어를 할줄몰라 친구혼자 준비할순없기에 블로그나 지식인의 도움을받을까합니다현재 비행기표부터 예매는 해두었고 1달쯤전 준비를 할려고합니다여행목적은 힐링이지만 저는 힐링보단 관광쪽으로 생각을 갖고있기에 관광위주가 많네요.여행의목적은 오사카쪽을 생각중이며 오사카여행의 정보가 가득한 블로그나 지식인분들이 있으시면 부탁드립니다우선 교통편은 비행기표 예매부터 했구요 호텔은 아직미정입니다1달째 준비중이다보니 머리속이 복잡하네요 ㅠ 오사카여행의 조언및 오사카여행을 다녀오신분들 블로그로라도 도움받고자 합니다.꼭좀 여행의 조언이나 도움정보들을 블로그로 써주시면 감사하겠습니다ㅠㅠㅠ ------------------------------------------------------------------------------------------------------ 질문자님의 글을 보니 저의 첫 자유여행이 생각나네요. 저도 처음엔 비행기 티켓, 숙박부터 예약하느라 좀 힘들었어요. 일본여행의 정보들은 일본관광청에 가시거나 서점에선 '100배 즐기기'시리즈를 많이 보시더라구요. 인터넷에선`

- 100step 학습

`### 질문: 일본 여행에 대해서 블로그를 작성해줘`

`### 답변: :### 블로그 댓글: ##### 블로그 본문:####################################################### #일본 후쿠오카여행 #유유버스 타고 #여행하자 #여행코스 #버스여행 #여행예약​안녕하세요~ 이웃님들^^​ ​이제 슬슬 여름 성수기라국내여행부터 해외여행까지준비 많이 하시는 시기죠?​저도 올해 해외여행 계획하면서지난 번 오사카 여행때 이용했던유유버스를 다시 이용하게 되어여러분들께 소개해드리려고 해요~​그럼 여행코스부터예약팁까지 자세히 알려드릴게요^^​ ​​ 후쿠오카 여행코스 알아보기​먼저 후쿠오카는 한국과 가까워서 비행시간도 짧아비행기로 다녀오기 괜찮은 곳이에요​저도 후쿠오카로 갈 때는 비행기,한국으로 돌아올때는 배타고 왔어요​저는 후쿠오카 공항-유후인-뱃부-유후인​이렇게 총 4일 일정이었는데요보통`